# Headway Explorer Tool

This tool is for exploring headway data given your specifications. You must specify desired service days, time period, and maximum headway length (in minutes). The resulting table flags the Bay Area transit service that meet the criteria you specified.


### TODO:

- add visualization of stops
- add route type (e.g. bus) and buffer selector tool

In [1]:
from widget_fns import *
from headway_fns import (pull_GTFS_tables,
                         calc_headways,
                         get_headway_calc_period_str)

# I. Get data:

## Pull static GTFS tables for headway calculation

In [2]:
# pull necessary GTFS tables from Redshift
df_dict = pull_GTFS_tables()

2019-04-26 16:18:27,704 INFO sqlalchemy.engine.base.Engine select version()
2019-04-26 16:18:27,705 INFO sqlalchemy.engine.base.Engine {}
2019-04-26 16:18:27,760 INFO sqlalchemy.engine.base.Engine select current_schema()
2019-04-26 16:18:27,761 INFO sqlalchemy.engine.base.Engine {}
2019-04-26 16:18:27,816 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-04-26 16:18:27,817 INFO sqlalchemy.engine.base.Engine {}
2019-04-26 16:18:27,845 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-04-26 16:18:27,845 INFO sqlalchemy.engine.base.Engine {}
2019-04-26 16:18:27,900 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2019-04-26 16:18:27,901 INFO sqlalchemy.engine.base.Engine {'name': 'SELECT * FROM transportation.gtfs_trips;'}
2019-04-26 16:18:27,984 INFO sqlalchemy.engi

# II. Get user input:

## Specify desired service days:

In [12]:
day_type_button, weekday_checkboxes = init_day_type_button()
display(day_type_button)

RadioButtons(description='Day type:', options=('Weekday', 'Weekend', 'Select Day(s)'), value='Weekday')

In [27]:
# save user-specified input (from above cells)
service_days = get_days(day_type_button, weekday_checkboxes)

you selected ['saturday', 'sunday']


## Specify desired time period:

In [14]:
# make dropdown
time_period_menu, custom_time_dict = init_time_period_menu()
display(time_period_menu)

Dropdown(description='Time period:', options=('AM Peak (6-10AM)', 'PM Peak (3-7PM)', 'Standard weekday (6AM-10…

In [28]:
start_time, end_time = get_time_period(time_period_menu, custom_time_dict)

You selected start time of 08:00:00 and end time of 22:00:00


## Specify desired maximum headway length (in minutes):

In [7]:
headway_selector, custom_headway = init_headway_selector()
display(headway_selector)

Dropdown(description='Headway:', options=('15 mins', '20 mins', '30 mins', 'Enter your own'), value='15 mins')

In [29]:
max_headway = get_headway(headway_selector, custom_headway)

you selected a headway of 30 mins


# III. Compute headways table

In [30]:
user_input_args = {'max_headway': max_headway,
                   'service_days': service_days,
                   'time_period': {'start_time_td': start_time,
                                   'end_time_td': end_time}}

headways_args = {**df_dict, **user_input_args}

calc_id = get_headway_calc_period_str(service_days, start_time, end_time)
headways_table = calc_headways(**headways_args)

# IV. Download and inspect data

In [31]:
show_download_button(headways_table, calc_id)

Button(description='Download Data', style=ButtonStyle())

In [26]:
# displayed table must have pagination and filtering 
headways_table.head()

,agency_id,agency_name,route_id,direction_id,trip_headsign,route_type,Route_Pattern_ID,Weekday_06:00-22:00_Headways,Weekday_06:00-22:00_Num_Trips,time_period,Meets_criteria
0,3D,Tri Delta Transit,388,Outbound,Eastbound Kaiser,3,3D-388-Outbound,50,19,06:00-22:00,0
1,3D,Tri Delta Transit,388,Outbound,Eastbound Kaiser,3,3D-388-Outbound,50,19,06:00-22:00,0
2,3D,Tri Delta Transit,388,Outbound,Eastbound Kaiser,3,3D-388-Outbound,50,19,06:00-22:00,0
3,3D,Tri Delta Transit,388,Outbound,Eastbound Kaiser,3,3D-388-Outbound,50,19,06:00-22:00,0
4,3D,Tri Delta Transit,388,Outbound,Eastbound Kaiser,3,3D-388-Outbound,50,19,06:00-22:00,0


## TODO: show route/stops visualization

- for mapping, add stops (join with route_pattern_id) (stop id, lat, long, Route_Pattern_ID -> draw routes on map)